In [26]:
#basic lib
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import re
from datetime import datetime
import os
import scipy as sci
import plotly
import stat
import sklearn as sk
import warnings
import time 
import requests
import logging

import plotly
import chart_studio.plotly as py #pip install chart_studio
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot, plot


warnings.filterwarnings("ignore")

In [27]:
CLIENT_ID = '87c5153b7f74400ab86057e17cd8974a'
CLIENT_SECRET = 'e69632e3ef7448fd847a181730fac26c'
username = 'fv9r6gystti1tplk7pwckqi0e'
redirect_uri = 'http://localhost:3005/'
scope = 'user-read-recently-played'

In [28]:
# spotify 
import spotipy #pip install spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

In [29]:
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=CLIENT_ID,   
                                   client_secret=CLIENT_SECRET,     
                                   redirect_uri=redirect_uri)

In [30]:
#https://open.spotify.com/artist/00sazWvoTLOqg5MFwC68Um?si=mQvbYg1OSuKUHdtBNj3UpA

In [72]:
import requests

def search_spotify_song(query):
    '''Performs a query on Spotify API to search for songs.'''
    
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {token}',
    }

    params = {
        'q': query,
        'type': 'track',
        'limit': 20# Limit the number of results to 3
    }

    try:
        response = requests.get('https://api.spotify.com/v1/search', headers=headers, params=params, timeout=5)
        response.raise_for_status()  # Raise HTTPError for bad responses

        data = response.json()
        tracks = data.get('tracks', {}).get('items', [])

        # Extracted data to be stored in a list of dictionaries
        extracted_data = []

        for track in tracks:
            track_data = {
                'Track Name': track['name'],
                'Artists': ', '.join(artist['name'] for artist in track['artists']),
                'duration': track['duration_ms'],
                'popularity': track['popularity'],
                'id': track['id']
            }
            extracted_data.append(track_data)

        return extracted_data

    except requests.exceptions.Timeout:
        print("Timeout error: The Spotify API request timed out.")
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    return None
# Example usage
query = 'Paint It Black' 


In [73]:
df = pd.DataFrame(search_spotify_song(query)) # duration_ms,popolarity

In [74]:
df

,Track Name,Artists,duration,popularity,id
0,Paint It Black,Claudillea,150592,40,0gfiNG4pWorM4prAHI3dV6
1,"Paint It, Black",The Rolling Stones,202266,85,63T7DJ1AFDD6Bn8VzG6JE8
2,Paint It Black,Wednesday Addams,145333,63,1OuCn2F9BmyTAdM0Jylo9X
3,"Paint It, Black",Ramin Djawadi,344017,49,6rSVuRt7xl4GsEQv4tphSl
4,"Paint It, Black",The Rolling Stones,224093,49,73SPjNZV2wVy8imEmckGYw
5,"Paint It, Black",Ciara,234146,57,2VP6E3tkWSbD2uHBioBDLp
6,Paint It Black - Epic Trailer Version,"Hidden Citizens, Rånya",230704,55,6hTFN4elhqitfZKhfERZU5
7,"Paint It, Black",The Rolling Stones,224133,0,3QfxJO5q8uajOfZO72pLqF
8,"Paint It, Black - (Original Single Mono Version)",The Rolling Stones,224133,56,7lY8aoN3wUR3NY4nUwigPv
9,"Paint It, Black - Cello Version",Diego Mitre,142164,39,16RnmmFTxycAY4eeowvGX8


In [60]:
df0 = df[0:5]
df0 = df0.append(df[])

In [75]:
def create_dataframe0(df):
    """Update a DataFrame with audio features for each track."""
    # Initialize columns for audio features
    audio_features_list = ['danceability', 'loudness', 'tempo', 'acousticness', 'energy', 'valence', 'liveness',
                           'speechiness', 'instrumentalness', 'time_signature','key', 'mode']
    for feature in audio_features_list:
        df[feature] = np.nan

    # Get track IDs
    track_ids = df['id'].tolist()

    # Make batch requests
    for i in range(0, len(track_ids), 100):
        batch = track_ids[i:i+100]
        audio_features_batch = sp.audio_features(batch)

        # Update rows with audio features
        for audio_features in audio_features_batch:
            if audio_features is not None:
                track_id = audio_features['id']
                row_index = df[df['id'] == track_id].index[0]
                df.loc[row_index, audio_features_list] = [audio_features[feature] for feature in audio_features_list]

    return df

In [76]:
df1 = create_dataframe0(df)


Couldn't write token to cache at: .cache


In [77]:
df1.describe(include='all')

,Track Name,Artists,duration,popularity,id,danceability,loudness,tempo,acousticness,energy,valence,liveness,speechiness,instrumentalness,time_signature,key,mode
count,20,20,20.000000,20.00000,20,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.0,20.000000,20.000000
unique,10,15,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Paint It Black,The Rolling Stones,NaN,NaN,0gfiNG4pWorM4prAHI3dV6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,6,5,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,207101.900000,43.35000,NaN,0.507300,-8.442650,130.004550,0.268933,0.632650,0.484400,0.194225,0.062105,0.296543,4.0,5.050000,0.300000
std,NaN,NaN,51696.117891,18.96333,NaN,0.113136,3.351778,29.083607,0.357609,0.257783,0.251779,0.119240,0.025474,0.413625,0.0,2.684752,0.470162
min,NaN,NaN,137142.000000,0.00000,NaN,0.259000,-15.376000,73.854000,0.000533,0.222000,0.076600,0.068800,0.031200,0.000000,4.0,0.000000,0.000000
25%,NaN,NaN,149277.250000,28.75000,NaN,0.471500,-10.407500,109.060750,0.013450,0.393500,0.234250,0.100475,0.037275,0.000991,4.0,4.750000,0.000000
50%,NaN,NaN,220379.500000,48.50000,NaN,0.504000,-7.927000,129.900000,0.123500,0.662000,0.530500,0.126000,0.062500,0.066250,4.0,5.000000,0.000000
75%,NaN,NaN,232046.000000,55.25000,NaN,0.560250,-7.024000,159.625500,0.308500,0.858250,0.730750,0.310750,0.090400,0.868500,4.0,6.250000,1.000000


work on features

In [78]:
replacement_mode = {1:'major',0:'minor'}
replacement_keys = {0:'C',1:'C#/Db',2:'D',3:'D#,Eb',4:'E',5:'F',6:'F#/Gb',7:'G',8:'G#/Ab',9:'A',10:'A#/Bb',11:'B',-1:'na'}
# # is sharp and b is flat 

In [79]:
df1['loudness'] = (df1['loudness'] - (-60)) / (0 - (-60))
df1['tempo'] = (df1['tempo'] - (250)) / (0 - (200))
df1['duration'] = round(df1['duration']/60000,1)
df1['key'] = df1['key'].replace(replacement_keys)
df1['mode'] = df1['mode'].replace(replacement_mode)

In [80]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track Name        20 non-null     object 
 1   Artists           20 non-null     object 
 2   duration          20 non-null     float64
 3   popularity        20 non-null     int64  
 4   id                20 non-null     object 
 5   danceability      20 non-null     float64
 6   loudness          20 non-null     float64
 7   tempo             20 non-null     float64
 8   acousticness      20 non-null     float64
 9   energy            20 non-null     float64
 10  valence           20 non-null     float64
 11  liveness          20 non-null     float64
 12  speechiness       20 non-null     float64
 13  instrumentalness  20 non-null     float64
 14  time_signature    20 non-null     float64
 15  key               20 non-null     object 
 16  mode              20 non-null     object 
dtyp

In [70]:
file_path = 'song_renditions.csv'
df1.to_csv(file_path,index=False)

In [80]:
attributes = ['danceability', 'loudness', 'energy','valence','speechiness',
              'liveness','acousticness', 'instrumentalness']

In [81]:
df_new = df1[attributes]

In [83]:
df_new

,danceability,loudness,energy,valence,speechiness,liveness,acousticness,instrumentalness
0,0.464,0.845550,0.795,0.6120,0.0926,0.399,0.049300,0.002440
1,0.539,0.878850,0.475,0.1820,0.0314,0.349,0.162000,0.000000
2,0.612,0.831133,0.319,0.7270,0.0312,0.103,0.892000,0.930000
3,0.626,0.872217,0.615,0.0795,0.0898,0.127,0.189000,0.000012
4,0.435,0.909033,0.631,0.2230,0.0451,0.111,0.102000,0.000000
5,0.543,0.743733,0.222,0.5600,0.0336,0.241,0.894000,0.927000
6,0.524,0.952233,0.941,0.4740,0.0577,0.298,0.000805,0.001640
